# **Projet Python : prévision de la production éléctrique éolienne en France**
#### Florian Basso, Théo Ferry, Julien Peignon
***
L'objectif de ce projet est d'utiliser des données météorologiques de vents ainsi que des données liées au parc éolien français (position des éoliennes, puissance potentielle, etc..) afin de mettre en place des modèles de prédiction de la production d'énergie éolienne.
***
## 1) Données
### a) Récupération
Les données du Ministère de la Transition Écologique et de RTE sont en open data. Les données de Météo France sur les vents sont disponibles avec une API.

**Code à venir:** API données métérologiques, code téléchargement données éoliennes + données RTE

### b) Travail sur nos bases de données

L’enjeu sera de fusionner nos trois bases de données, de les retravailler de manière à associer les vents (qui sont mesurés à une localisation précise) aux éoliennes qui produisent dans un rayon proche afin d’estimer le plus finement possible la production en fonction des conditions météorologiques locales.

### c) Analyse descriptive

Premièrement, nous allons créer une carte afin de visualiser la position des installations éoliennes sur le territoire français

In [ ]:
!pip install matplotlib
!pip install geopandas
!pip install pandas  

In [10]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [11]:
url = 'https://opendata.agenceore.fr/explore/dataset/installations-de-production-eolien-par-commune/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false'

## 2) Modélisation
### a) Modèle linéaire simple

Nous commencerons la modélisation pour une prédiction « naïve » de la production électrique avec un modèle de régression linéaire simple.

### b) Machine learning

Nous utilisons ensuite des modèles de machine learning plus performants pour d’avoir des prédictions plus précises. Nous pourrons comparer les résultats des deux modèles afin d’illustrer les faiblesses du modèle linéaire simple.